In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline
plt.rcParams['figure.figsize'] = (16,10)

In [2]:
def read_csv(TRAIN_PATH):
    chunksize = 5_000_000 # batch size
    
    traintypes = {
        'fare': 'float32',
        'from': 'str',
        'to': 'str',
        'p_count': 'uint8', 
        'hour': 'uint8',
        'weekday': 'uint8',
        'week': 'uint8',
        'year': 'int16',
        'dist_e': 'float32',
        'dist_t': 'float32',
        'x0': 'float32',
        'x1': 'float32',
        'y0': 'float32',
        'y1': 'float32',
    }
    
    cols = list(traintypes.keys())
    
    # loading the dataframe into list of small dataframes
    df_list = []

    for df_chunk in pd.read_csv(TRAIN_PATH, usecols=cols, dtype=traintypes, chunksize=chunksize):
        df_list.append(df_chunk) 
        df = pd.concat(df_list)
        
    return df

In [3]:
train = read_csv('../input/train_cleaned_2.csv')

In [4]:
train = train[(train['from'] != 'Manhattan') & (train['to'] != 'Manhattan')]

In [5]:
train.shape

(1823399, 14)

In [6]:
y = train['fare']
X = train.drop(['fare'], axis=1)
del train

from sklearn.model_selection import train_test_split
X_train, X_eval, y_train, y_eval = train_test_split(X, y, test_size=0.25, random_state=379582)

del X,y

In [7]:
import tensorflow as tf

In [8]:
locations = [
    'Manhattan', 'JFK', 'LaGardia', 'Newark', 'Teterboro', 'Brooklin', 'Bronx', 'Staten', 'Queens', 'Westchester',
    'Nassau', 'Jersey', 'Bayonne', 'WStaton', 'West', 'WNewYork', 'North'        
]

# num_buckets = [0, num_buckets)
p_count = tf.feature_column.numeric_column('p_count')
taxicab = tf.feature_column.numeric_column('dist_t')
euclidean = tf.feature_column.numeric_column('dist_e')

x0 = tf.feature_column.numeric_column('x0')
x1 = tf.feature_column.numeric_column('x1')
y0 = tf.feature_column.numeric_column('y0')
y1 = tf.feature_column.numeric_column('y1')

numbuckets = 30
dx = np.linspace(-74.27, -72.98, numbuckets).tolist()
dy = np.linspace(40.56, 41.71, numbuckets).tolist()

x0_B = tf.feature_column.bucketized_column(x0, dx)
x1_B = tf.feature_column.bucketized_column(x1, dx)
y0_B = tf.feature_column.bucketized_column(y0, dy)
y1_B = tf.feature_column.bucketized_column(y1, dy)

xy_0 = tf.feature_column.crossed_column([x0_B, y0_B], numbuckets * numbuckets)
xy_1 = tf.feature_column.crossed_column([x1_B, y1_B], numbuckets * numbuckets)
xy_pair = tf.feature_column.crossed_column([xy_0, xy_1], numbuckets**4)

loc_from = tf.feature_column.categorical_column_with_vocabulary_list('from', locations)
loc_to = tf.feature_column.categorical_column_with_vocabulary_list('to', locations)

# definition of the categorical columns
hour = tf.feature_column.categorical_column_with_identity('hour', num_buckets = 24)
weekday = tf.feature_column.categorical_column_with_identity('weekday', num_buckets = 7)
week = tf.feature_column.categorical_column_with_identity('week', num_buckets = 54)
year = tf.feature_column.categorical_column_with_vocabulary_list('year', [2009, 2010, 2011, 2012, 2013, 2014, 2015])

from_to = tf.feature_column.crossed_column([loc_from, loc_to], len(locations)**2)
from_to_year = tf.feature_column.crossed_column([from_to, year], len(locations)**2 * 7)
from_to_year_week = tf.feature_column.crossed_column([from_to_year, week], len(locations)**2 * 7 * 54)

# definition of the feature crosses
hour_weekday = tf.feature_column.crossed_column([hour, weekday], 24 * 7)
weekday_week_year = tf.feature_column.crossed_column([weekday, week, year], 7 * 54 * 7)
week_year = tf.feature_column.crossed_column([week, year], 54 * 7)
weekday_year = tf.feature_column.crossed_column([weekday, year], 7 * 7)
hour_weekday_year = tf.feature_column.crossed_column([hour, weekday, year], 24 * 7 * 7)

In [9]:
deep_columns = [
    # Embedding_column to "group" together ...
    tf.feature_column.embedding_column(hour_weekday, 50),
    tf.feature_column.embedding_column(weekday_week_year, 50),
    tf.feature_column.embedding_column(week_year, 50),
    tf.feature_column.embedding_column(weekday_year, 50),
    tf.feature_column.embedding_column(hour_weekday_year, 50),
    tf.feature_column.embedding_column(xy_pair, 50),
    tf.feature_column.embedding_column(from_to, 50),
    tf.feature_column.embedding_column(from_to_year, 50),
    tf.feature_column.embedding_column(from_to_year_week, 50),

    # indicator columns
    tf.feature_column.indicator_column(hour),
    tf.feature_column.indicator_column(weekday),
    tf.feature_column.indicator_column(week),
    tf.feature_column.indicator_column(year),
    tf.feature_column.indicator_column(xy_0),
    tf.feature_column.indicator_column(xy_1),
    tf.feature_column.indicator_column(loc_from),
    tf.feature_column.indicator_column(loc_to),

    # numeric columns
    taxicab, euclidean, p_count
]

In [10]:
def train_input_fn(features, labels, batch_size):
    return tf.estimator.inputs.pandas_input_fn(
        x = features,
        y = labels,
        num_epochs = 1,
        shuffle = True,
        batch_size = batch_size)

def eval_input_fn(features, labels):
    return tf.estimator.inputs.pandas_input_fn(
        x = features,
        y = labels,
        num_epochs = 1,
        shuffle = True)

def pred_input_fn(features):
    return tf.estimator.inputs.pandas_input_fn(
        x = features,
        num_epochs = 1,
        shuffle = False)

In [11]:
model_dir = './DNN_Regressor_other'

file_writer = tf.summary.FileWriter(model_dir)

estimator = tf.estimator.DNNRegressor(
    model_dir = model_dir,
    feature_columns = deep_columns,
    optimizer = tf.train.AdamOptimizer(learning_rate=0.0001),
    hidden_units = [128,128,128,128], 
    batch_norm = True,
)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './DNN_Regressor_other', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001660D292EF0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [12]:
train_spec = tf.estimator.TrainSpec(input_fn = train_input_fn(X_train, y_train, batch_size=2500))
eval_spec = tf.estimator.EvalSpec(input_fn = eval_input_fn(X_eval, y_eval))

In [15]:
for i in range(5):
    tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)

INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after every checkpoint. Checkpoint frequency is determined based on RunConfig arguments: save_checkpoints_steps None or save_checkpoints_secs 600.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./DNN_Regressor_other\model.ckpt-3288
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 3288 into ./DNN_Regressor_other\model.ckpt.
INFO:tensorflow:loss = 104533.45, step = 3288
INFO:tensorflow:global_step/sec: 6.54138
INFO:tensorflow:loss = 122265.57, step = 3388 (15.288 sec)
INFO:tensorflow:global_step/sec: 7.21132
INFO:tensorflow:loss = 99859.164, step = 3488 (13.867 sec)
INFO:tensorflow:global_step/sec: 7.11938
INFO:tensorflow:los

INFO:tensorflow:global_step/sec: 7.25748
INFO:tensorflow:loss = 133759.34, step = 5332 (13.779 sec)
INFO:tensorflow:global_step/sec: 7.34649
INFO:tensorflow:loss = 92584.3, step = 5432 (13.612 sec)
INFO:tensorflow:Saving checkpoints for 5480 into ./DNN_Regressor_other\model.ckpt.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-09-05-00:54:18
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./DNN_Regressor_other\model.ckpt-5480
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation [10/100]
INFO:tensorflow:Evaluation [20/100]
INFO:tensorflow:Evaluation [30/100]
INFO:tensorflow:Evaluation [40/100]
INFO:tensorflow:Evaluation [50/100]
INFO:tensorflow:Evaluation [60/100]
INFO:tensorflow:Evaluation [70/100]
INFO:tensorflow:Evaluation [80/100]
INFO:tensorflow:Evaluation [90/100]
INFO:tensorflow:Evaluation [100/100]
INFO:tensorflow:Finished e

In [16]:
infer = pd.read_csv('../input/test_processed.csv', index_col='key')

generator = estimator.predict(input_fn=pred_input_fn(infer))
predictions = [next(generator) for i in range(len(infer))]
values = [val['predictions'].tolist()[0] for val in predictions]

ids = infer.index
submission = pd.DataFrame()
submission['fare_amount'] = values
submission['key'] = ids
submission.set_index('key', inplace=True)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./DNN_Regressor_other\model.ckpt-6028
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [18]:
submission2 = pd.read_csv('../output/20.dnn_regressor_manhattan_other_best_rmse_(2.43+3.01).csv', index_col='key')

In [19]:
mht = infer[(infer['from'] == 'Manhattan') & (infer['to'] == 'Manhattan')].index

In [20]:
submission.loc[mht, 'fare_amount'] = submission2.loc[mht, 'fare_amount']

In [21]:
submission.to_csv('../output/21.dnn_regressor_manhattan_other_dnn_regressor_rmse_(2.43+9.36).csv')